---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [7]:
import pandas as pd
import numpy as np

def blight_model():
    #! cat readonly/train.csv > train.csv
    #! cat readonly/test.csv > test.csv
    # run this without comments once run with readonly and submit without readonly commenting above
    
    
    # Your code here
    X_train=pd.read_csv('train.csv',encoding='latin1')
    X_test=pd.read_csv('test.csv',encoding='latin1')
    X_train=X_train.drop(['payment_amount','payment_date','payment_status','balance_due','collection_status',
                         'compliance_detail'],axis=1)
    #a=pd.notnull(X_train['compliance'])
    X_train=X_train[pd.notnull(X_train['compliance'])]
    y_train=X_train['compliance']
    y_train.reset_index(drop=True, inplace=True)
    #X_train=X_train[X_train['compliance']!= np.nan]
    #X_train.head(20)# Your answer here
    address=pd.read_csv('addresses.csv')
    loc=pd.read_csv('latlons.csv')
    #merging not mapping 
    a=address.merge(loc,how='inner',on='address')
    X_train=X_train.merge(a,how='inner',on='ticket_id')
    X_test=X_test.merge(a,how='inner',on='ticket_id')
    #X_train['ticket_id']=X_train['ticket_id'].map(address)
    #X_test.isnull().sum() using this can see numbers of entries that are empty in column drop if greater than 50%
    X_test=X_test.drop(['inspector_name', 'violator_name', 'violation_street_number', 
                      'violation_street_name', 'violation_zip_code', 'mailing_address_str_number',
                     'mailing_address_str_number', 'mailing_address_str_name', 'zip_code',
                     'non_us_str_code', 'country', 'violation_code', 'city',
                    'violation_description', 'clean_up_cost', 'address', 'grafitti_status'],axis=1)
    X_train=X_train.drop(['inspector_name', 'violator_name', 'violation_street_number', 
                      'violation_street_name', 'violation_zip_code', 'mailing_address_str_number',
                     'mailing_address_str_number', 'mailing_address_str_name', 'zip_code',
                     'non_us_str_code', 'country', 'violation_code', 'city',
                    'violation_description','clean_up_cost', 'address', 'grafitti_status'],axis=1)
    
    X_test['lat']=X_test['lat'].fillna(X_test['lat'].mode()[0])
    X_test['lon']=X_test['lon'].fillna(X_test['lon'].mode()[0])
    X_train['lat']=X_train['lat'].fillna(X_train['lat'].mode()[0])
    X_train['lon']=X_train['lon'].fillna(X_train['lon'].mode()[0])
    X_train['lon']=X_train['lon'].fillna(X_train['lon'].mode()[0])
    X_train['state']=X_train['state'].fillna(X_train['state'].mode()[0])
    X_test['state']=X_test['state'].fillna(X_test['state'].mode()[0])
    X_train['agency_name']=X_train['agency_name'].fillna(X_train['agency_name'].mode()[0])
    X_test['agency_name']=X_test['agency_name'].fillna(X_test['agency_name'].mode()[0])
    X_test['disposition']=X_test['disposition'].fillna(X_test['disposition'].mode()[0])
    X_train['disposition']=X_train['disposition'].fillna(X_train['disposition'].mode()[0])
    X_train=X_train[['ticket_id', 'fine_amount', 'admin_fee','state_fee','late_fee','discount_amount','judgment_amount','lat','lon','agency_name','state','disposition']]
    X_test=X_test[['ticket_id', 'fine_amount','admin_fee','state_fee','late_fee','discount_amount','judgment_amount','lat','lon','agency_name','state','disposition']]
    
    X_test=X_test.set_index('ticket_id')
    X_train=X_train.set_index('ticket_id')
    
    X1=pd.concat([X_train,X_test])# combining both test and test for adding dummies not present in test train vice versa for higher score 
    X1=pd.get_dummies(X1,columns=['agency_name', 'state', 'disposition'])
    X_train_final=X1.iloc[:len(X_train),:]
    X_test_final=X1.iloc[len(X_train):,:]
    
    #when multiple use inside list
    #from sklearn.preprocessing import MinMaxScaler
    #from sklearn.linear_model import LinearRegression
    #from sklearn.model_selection import GridSearchCV
    #from sklearn.metrics import roc_auc_score, confusion_matrix
    #scaler=MinMaxScaler()
    #X_train_scaled=scaler.transform(X_train_final)
    #X_test_scaled=scaler.transform(X_test_final)
    #clf=LinearRegression()
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LogisticRegression# unlike linear it is for binary variable not continuos
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import roc_auc_score
    scaler=MinMaxScaler()
    X_train_scaled=scaler.fit_transform(X_train_final)
    X_test_scaled=scaler.fit_transform(X_test_final)
    clf=LogisticRegression()
    #grid_values={'C':np.logspace(-1,4,6)} {C:[]}
    
    #grid_values={'C':np.logspace(-1,4,6)} best_params_ give c=10000
    grid_values={'C':[1000]}
    grid_clf_auc=GridSearchCV(clf,param_grid=grid_values,scoring='roc_auc')
    #grid_clf_auc.fit(X_train_scaled,y_train)
    
    prob = grid_clf_auc.fit(X_train_scaled,y_train).predict_proba(X_test_scaled)
    prob1=[p[1] for p in prob]
    #import pickle 
    #saved_model = pickle.dumps(grid_clf_auc)
    #gcv_from_pickle = pickle.loads(saved_model)
    #gcv_from_pickle.predict(X_test) 
    #gcv_from_pickle.predict_proba(X_test_scaled) 


    #return prob#grid_clf_auc.score(X_train_scaled,y_train)# gives score of  0.79008295392057215
    #prob = grid_clf_auc.predict_proba(X_test_scaled)
    result = pd.Series(prob1,index=X_test_final.index)
    #return print(grid_clf_auc.best_params_) gives {'C': 10000.0}
    #return result#len(prob1)#result 
    return result#len of result is 61001 as demanded
   

    

    
    #return X_train_final#.head(10)

In [8]:
blight_model()


/bin/sh: 1: cannot create train.csv: Read-only file system


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.003725
285362    0.000632
285361    0.002194
285338    0.020708
285346    0.024125
285345    0.021071
285347    0.025809
285342    0.243639
285530    0.005062
284989    0.005962
285344    0.023725
285343    0.006479
285340    0.007059
285341    0.025809
285349    0.018656
285348    0.016283
284991    0.007133
285532    0.032098
285406    0.002112
285001    0.007748
285006    0.003892
285405    0.000508
285337    0.000301
285496    0.006325
285497    0.005145
285378    0.000470
285589    0.001119
285585    0.002330
285501    0.006274
285581    0.000536
            ...   
376367    0.001107
376366    0.002210
376362    0.013372
376363    0.014317
376365    0.001107
376364    0.002210
376228    0.007668
376265    0.002111
376286    0.010826
376320    0.005352
376314    0.005102
376327    0.066335
376385    0.019562
376435    0.209143
376370    0.144900
376434    0.003291
376459    0.012165
376478    0.000014
376473    0.005117
376484    0.006918
376482    0.001489
37